In [47]:
from datasets import load_dataset

df = load_dataset('amphora/natural_reasoning_nonmath_300k',split='train').to_pandas()
# df = df[df.category.str.contains('/')]
# df = df[df.category.str.contains('Science')]
df = df[df.category=='Science/Physics']
df = df.sample(1500)

In [38]:
qrys = []
for _,row in df.iterrows():
    from prompts import score_question
    score_question += f"\n\n {row.prompt}"
    package = {
    "custom_id": f"request-{_}", 
    "method": "POST", 
    "url": "/v1/chat/completions", 
    "body": {
        "model": "gpt-4o", 
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content":score_question}
        ],
        # "max_tokens": 1000
    }}
    qrys.append(package)

import json

# Open a file in write mode
with open("batch/score_1000_qry.jsonl", "w") as f:
    for entry in qrys:
        # Write the dictionary as a JSON string followed by a newline
        f.write(json.dumps(entry) + "\n")

In [40]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

batch_input_file = client.files.create(
    file=open("batch/score_1000_qry.jsonl", "rb"),
    purpose="batch"
)

batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "nightly eval job"
    }
)

In [41]:
batch = client.batches.retrieve('batch_67fcb5fb05008190a9220ed130c8797a')
print(batch)

Batch(id='batch_67fcb5fb05008190a9220ed130c8797a', completion_window='24h', created_at=1744614907, endpoint='/v1/chat/completions', input_file_id='file-Rq4x8PxKRYfQuwRxEGSVXp', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1744701307, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [46]:
from src import download_openai_file

download_openai_file(
    'file-HZKPnPWSiaZyzsF6PBd6Y1',
    openai_api_key,
    "batch/score_1000_qry_output.jsonl"
)

File downloaded successfully and saved as 'batch/score_1000_qry_output.jsonl'.


In [48]:
from src import load_jsonl_file

output = load_jsonl_file('batch/score_1000_qry_output.jsonl')

final = []
for item in output:
    idx = item['custom_id'].split('-')[1]
    prompt, reference, category, llama, _ = df[df.index==int(idx)].values[0]
    judge = item['response']['body']['choices'][0]['message']['content']
    final.append(
        [prompt, reference, category, llama, judge]
    )

import pandas as pd
df = pd.DataFrame(final,columns = ['o_prompt','o_reference','o_category','o_response','judge'])

In [34]:

qrys = []
for _,row in df.iterrows():
    from prompts import score_question,refine_question
    score_question += f"\n\n {row.o_prompt}"
    package = {
    "custom_id": f"request-{_}", 
    "method": "POST", 
    "url": "/v1/chat/completions", 
    "body": {
        "model": "gpt-4o", 
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content":score_question},
            {"role": "assistant", "content":row.judge},
            {"role": "user", "content":refine_question}
        ],
        # "max_tokens": 1000
    }}
    qrys.append(package)

import json

# Open a file in write mode
with open("batch/score_1000_refine.jsonl", "w") as f:
    for entry in qrys:
        # Write the dictionary as a JSON string followed by a newline
        f.write(json.dumps(entry) + "\n")

In [36]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

batch_input_file = client.files.create(
    file=open("batch/score_1000_refine.jsonl", "rb"),
    purpose="batch"
)

batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "nightly eval job"
    }
)

Batch(id='batch_67fcc1238ab081909e3493c7efdec8d1', completion_window='24h', created_at=1744617763, endpoint='/v1/chat/completions', input_file_id='file-J51Q5hPat5EZGBLE9bbuEi', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1744704163, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [38]:
batch = client.batches.retrieve('batch_67fcc1238ab081909e3493c7efdec8d1')
print(batch)

Batch(id='batch_67fcc1238ab081909e3493c7efdec8d1', completion_window='24h', created_at=1744617763, endpoint='/v1/chat/completions', input_file_id='file-J51Q5hPat5EZGBLE9bbuEi', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1744618178, error_file_id=None, errors=None, expired_at=None, expires_at=1744704163, failed_at=None, finalizing_at=1744618055, in_progress_at=1744617765, metadata={'description': 'nightly eval job'}, output_file_id='file-35RuGYFctFkQrvPuEVRjSm', request_counts=BatchRequestCounts(completed=1500, failed=0, total=1500))


In [41]:
download_openai_file(
    'file-35RuGYFctFkQrvPuEVRjSm',
    openai_api_key,
    "batch/score_1000_refine_output.jsonl"
)

File downloaded successfully and saved as 'batch/score_1000_refine_output.jsonl'.


In [61]:
from src import load_jsonl_file

data = load_jsonl_file("batch/score_1000_refine_output.jsonl")

final = []
for item in data:
    try:
        idx = item['custom_id'].split('-')[1]
        prompt, reference, category, llama, judge = df[df.index==int(idx)].values[0]
        refined = item['response']['body']['choices'][0]['message']['content']
        refined = refined.split('[Refined Question]')[1].strip()
        final.append(
            [prompt, reference, category, llama, judge,refined]
        )
    except:
        continue

In [64]:
df = pd.DataFrame(final,columns=['o_prompt','o_reference','o_category','o_response','judge','r_prompt'])

In [68]:
df.to_parquet('snr_prompts_20250414.parquet',index=False)